## **DimUser -Table**

In [0]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(project_root)

In [0]:
# Importando librerias
from pyspark.sql.functions import *
from pyspark.sql.types import *
from utils.transformations import reusable

In [0]:
from utils.transformations import reusable

In [0]:
df = spark.read.format("parquet")\
               .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/DimUser")

#### **AUTOLOADER**

In [0]:
schema_path = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimUser/schema"
checkpoint_path = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimUser/checkpoint"

df_user = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format","parquet")\
            .option("cloudFiles.schemaLocation",schema_path)\
            .option("schemaEvolveMode","addNewColumns")\
            .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/DimUser")

In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumn(df_user,['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])

In [0]:
df_user.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation",checkpoint_path)\
                   .trigger(once=True)\
                   .option("path","abfss://silver@staccountazureproject.dfs.core.windows.net/DimUser/data")\
                   .toTable("spotify_cata.silver.DimUser")

## **DimArtist -Table**

In [0]:
schema_art = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimArtist/schema"
checkpoint_art = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimArtist/checkpoint"

df_art = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format","parquet")\
            .option("cloudFiles.schemaLocation",schema_art)\
            .option("schemaEvolveMode","addNewColumns")\
            .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/DimArtist")

In [0]:
df_art_onj = reusable()

df_art = df_art_onj.dropColumn(df_art,['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])

In [0]:
df_art.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation",checkpoint_art)\
                   .trigger(availableNow=True)\
                   .option("path","abfss://silver@staccountazureproject.dfs.core.windows.net/DimArtist/data")\
                   .toTable("spotify_cata.silver.DimArtist")

## **DimTrack -Table**

In [0]:
schema_track = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimTrack/schema"
checkpoint_track = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimTrack/checkpoint"

df_track = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format","parquet")\
            .option("cloudFiles.schemaLocation",schema_track)\
            .option("schemaEvolveMode","addNewColumns")\
            .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/DimTrack")

In [0]:
df_track = df_track.withColumn("durationFlg",when(col('duration_sec')<150, "low")\
                                            .when(col('duration_sec')<300, "medium")\
                                            .otherwise("high"))


df_track = df_track.withColumn("track_name", regexp_replace(col("track_name"),'-',' '))

df_track = reusable().dropColumn(df_track,['_rescued_data'])

In [0]:
df_track.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation",checkpoint_track)\
                   .trigger(availableNow=True)\
                   .option("path","abfss://silver@staccountazureproject.dfs.core.windows.net/DimTrack/data")\
                   .toTable("spotify_cata.silver.DimTrack")

## **DimDate -Table**

In [0]:
schema_date = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimDate/schema"
checkpoint_date = "abfss://silver@staccountazureproject.dfs.core.windows.net/DimDate/checkpoint"

df_date = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format","parquet")\
            .option("cloudFiles.schemaLocation",schema_date)\
            .option("schemaEvolveMode","addNewColumns")\
            .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/DimDate")

In [0]:
df_date = reusable().dropColumn(df_date,['_rescued_data'])

In [0]:
df_date.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation",checkpoint_date)\
                   .trigger(availableNow=True)\
                   .option("path","abfss://silver@staccountazureproject.dfs.core.windows.net/DimDate/data")\
                   .toTable("spotify_cata.silver.DimDate")

## **FactStream -Table**

In [0]:
schema_fact = "abfss://silver@staccountazureproject.dfs.core.windows.net/FactStream/schema"
checkpoint_fact = "abfss://silver@staccountazureproject.dfs.core.windows.net/FactStream/checkpoint"

df_fact = spark.readStream.format("cloudFiles")\
            .option("cloudFiles.format","parquet")\
            .option("cloudFiles.schemaLocation",schema_fact)\
            .option("schemaEvolveMode","addNewColumns")\
            .load("abfss://bronze@staccountazureproject.dfs.core.windows.net/FactStream")

In [0]:
df_fact = reusable().dropColumn(df_fact,['_rescued_data'])

In [0]:
df_fact.writeStream.format("delta")\
                   .outputMode("append")\
                   .option("checkpointLocation",checkpoint_fact)\
                   .trigger(availableNow=True)\
                   .option("path","abfss://silver@staccountazureproject.dfs.core.windows.net/FactStream/data")\
                   .toTable("spotify_cata.silver.FactStream")